In [15]:
import pandas as pd 
import numpy as np

In [16]:
BASE_MODEL_FOLDER = '../results_ensemble'
MODEL_LIST = [
    'AE',
    'ALS_ensemble',
    'PNCF',
    'SVDpp_ensemble'
]

In [17]:
def rmse(x,y):
    return np.sqrt(np.mean((x-y)**2))

In [18]:
val_truth = []
for i in range(0, 5):
    val_truth.append(pd.read_csv(f'../data_val_train_kfold/partition_{i}_val.csv')['Prediction'].to_numpy())

In [19]:
acc = []
for m in MODEL_LIST:
    tmp = []
    
    for i in range(0,5):
        df = pd.read_csv(f'{BASE_MODEL_FOLDER}/{m}/{m}_split_{i}_val_results.csv')
        t = df['Prediction'].to_numpy()
        score = rmse(t, val_truth[i])
        tmp.append(score)
    
    res = pd.DataFrame(index=[m])
    tmp = np.array(tmp)
    res['mean'] = np.mean(tmp)
    res['std'] = np.std(tmp)

    acc.append(res)
print(pd.concat(acc))


                    mean       std
AE              0.983427  0.002613
ALS_ensemble    0.980297  0.002219
PNCF            0.987562  0.002001
SVDpp_ensemble  0.973865  0.002502


In [20]:
def create_val_matrix(split):
    tmp = []
    for m in MODEL_LIST:
        df = pd.read_csv(f'{BASE_MODEL_FOLDER}/{m}/{m}_split_{split}_val_results.csv')
        t = df['Prediction'].to_numpy()

        tmp.append(t)
    return np.column_stack(tmp)


val_splits = []
for i in range(0, 5):
    val_splits.append(create_val_matrix(i))

def combine_models(yhat, coeff):
    coeff = np.array(coeff)
    return np.matmul(yhat, coeff)

In [21]:
tmp = []
for val_on  in range(0, 5):
    a = []
    for i in range(0,5):
        if i != val_on:
            coeffs = np.linalg.lstsq(val_splits[i], val_truth[i])[0]
            a.append(coeffs)
    coeffs = np.column_stack(a).mean(axis=1)
    
    res = combine_models(val_splits[val_on], coeffs)
    tmp.append(rmse(res, val_truth[val_on]))

res = pd.DataFrame(index=['Final ensemble'])
tmp = np.array(tmp)
res['mean'] = np.mean(tmp)
res['std'] = np.std(tmp)
acc.append(res)
print(pd.concat(acc))


                    mean       std
AE              0.983427  0.002613
ALS_ensemble    0.980297  0.002219
PNCF            0.987562  0.002001
SVDpp_ensemble  0.973865  0.002502
Final ensemble  0.970985  0.002393


/tmp/ipykernel_248008/1501064865.py:6: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  coeffs = np.linalg.lstsq(val_splits[i], val_truth[i])[0]
